In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import matplotlib as mpl

# dataset
dataset = "ImageNet"
# model
model = "ResNet18"
# excel file path
file_path = f"{dataset}_{model}-epoch_Results.xlsx"
# load sheet names
xls = pd.ExcelFile(file_path)
sheet_name = xls.sheet_names
sheet_name

In [ ]:
# dictionary to store Val Accuracies
Val_Accs = {}
Val_Loss = {}
LR_dict = {}
min_losses = []
best_epochs = []

# loop through each sheet
for sheet in sheet_name:
    df = pd.read_excel(file_path, sheet_name=sheet)
    cleaned_sheet = sheet.replace(model, "").replace("(", "").replace(")", "").strip()
    # filter val acc column
    if "Validation Accuracy" in df.columns:
        # extract the acc, loss and LR lists
        val_acc_list = df["Validation Accuracy"].dropna().tolist()
        val_loss_list = df["Validation Loss"].dropna().tolist()
        lr = df["LR"].dropna().tolist()
        # concatonate the lists into dictionary
        Val_Accs[cleaned_sheet] = val_acc_list
        Val_Loss[cleaned_sheet] = val_loss_list
        LR_dict[cleaned_sheet] = lr
        # find the best model and its epoch - to find the end point for plotting
        min_loss = min(val_loss_list)
        min_losses.append(min_loss)
        best_epoch = val_loss_list.index(min_loss) + 1
        best_epochs.append(best_epoch)
        # print the information
        print(f"Case: {cleaned_sheet}, Min Loss: {min_loss}, # Epochs: {best_epoch}")
    if "KE Validation Accuracy" in df.columns:
        Val_Accs[cleaned_sheet] = df["KE Validation Accuracy"].dropna().tolist()

# Find the LR with max length
jumps = []
# # last_epoch = min(best_epochs)
max_epoch = max(best_epochs)
last_epoch = int(sum(best_epochs)/4)
# last_epoch = best_epochs[-1]
best_lr = LR_dict[list(Val_Accs.keys())[best_epochs.index(max_epoch)]][:max_epoch]
# find the sudden chnages of LR
for i in range(len(best_lr)-1):
    if best_lr[i] != best_lr[i+1]:
        # print(best_lr[i+1], i+1+1)
        jumps.append(i+1+1)

jumps

In [ ]:
# Plotting Configuration
mpl.rcParams.update({
    "font.family": "sans-serif",
    "font.sans-serif": ["Arial", "Helvetica"],
    "font.size": 24,
    "text.usetex": False,
    "axes.titlesize": 40,
    "axes.labelsize": 50,
    "legend.fontsize": 40,
    "xtick.labelsize": 32.5,
    "ytick.labelsize": 32.5
})

sns.set_style("whitegrid")
color_palette = sns.color_palette("colorblind")

# Updated Plotting
plt.figure(figsize=(18, 9))

for idx, (label, val_acc_list) in enumerate(Val_Accs.items()):
    epochs = list(range(1, len(val_acc_list) + 1))
    # Clip to last_epoch
    if len(epochs) >= last_epoch:
        epochs = epochs[:last_epoch]
        val_acc_list = val_acc_list[:last_epoch]

    plt.plot(
        epochs, val_acc_list,
        label=label,
        color=color_palette[idx % len(color_palette)],
        marker='o',
        linewidth=4,
        markersize=8,
        linestyle='-'
    )

# Axis Labels and Title
plt.xlabel("# epochs", labelpad=10)
plt.ylabel("Val Acc (%)", labelpad=10)
# plt.ylabel("Validation Accuracy (%)", labelpad=10)
# plt.title(f"{model} - {dataset}", pad=15)

# Add vertical lines at LR jumps
for jump_epoch in jumps:
    if jump_epoch <= max_epoch:
        plt.axvline(x=jump_epoch, color='gray', linestyle='--', linewidth=2.25, alpha=0.65)

# Customize Grid
plt.grid(visible=True, which='major', axis='both', linestyle='-', linewidth=0.5, alpha=0.4)

# Legend and Layout
plt.legend(loc='lower right', frameon=True)
plt.xlim(1, last_epoch)
plt.tight_layout()

# Save as Vector PDF
plt.savefig(f"{dataset}_{model}_Validation_Accuracy_Plot.pdf", format='pdf', dpi=300)
plt.show()